In [31]:
import pandas as pd
import numpy as np
from pathlib import Path

# import own functions
from fill_and_map import map_econ_sdg, map_inds_sdg, fill_econ, fill_inds_trans

In [48]:
# get data
fpath = Path('data_raw/SDG-DEA_indicators_V5_20200708.xlsx')
inds = fill_inds_trans(fpath)
inds_econ = fill_econ(fpath)
inds_map = map_inds_sdg(fpath)
econ_map = map_econ_sdg(fpath)

# match index of economic indicators to SDG indicators, since some countries were exluded due to missing data
inds_econ = inds_econ.loc[inds.index, :]

In [49]:
# remove indicators we want to exclude from analysis
drop_inds = ['sdg17_govex', 'sdg10_fsi7']
inds = inds.drop(columns=drop_inds)
inds_map = {k:[ii for ii in v if ii not in drop_inds] for k, v in inds_map.items()}

In [56]:
inds

,sdg1_wpc,sdg1_320pov,sdg1_11_13_affected,sdg2_undernsh,sdg2_foodinsec,sdg2_stunting,sdg2_wasting,sdg2_obesity,sdg2_crlyld,sdg2_nue,...,sdg16_safe,sdg16_prs,sdg16_u5reg,sdg16_cpi,sdg16_weaponsexp,sdg16_rsf,sdg17_taxhaven,sdg9_intuse,sdg17_govrevGDP,sdg17_remittance
AFG,58.742400,34.19440,63.316473,69.7,45.694019,59.10,90.5,5.5,1.9817,0.522790,...,12.533762,3.680363,42.3,16.0,0.000000,62.72,0,11.447688,12.185672,95.850087
AGO,79.590000,57.99000,95.363706,76.1,59.216414,62.40,95.1,8.2,0.9347,0.469079,...,46.214709,2.424251,25.0,19.0,0.000000,61.65,0,14.339079,16.360591,99.998507
ALB,99.730000,96.95000,98.626170,94.5,61.441369,76.90,90.6,21.7,4.7164,0.425978,...,65.933543,3.600127,98.6,36.0,0.000000,70.51,0,71.847041,25.722581,90.344579
ARE,99.574048,99.23619,99.998058,97.5,90.514157,97.42,99.3,31.7,21.4874,0.196645,...,89.971806,5.857982,100.0,70.0,0.680841,59.14,0,94.819923,3.566406,99.109451
ARG,99.360000,97.40000,99.298629,96.2,67.896269,91.80,98.8,28.3,5.0965,1.056747,...,39.313905,3.613972,99.5,40.0,0.000000,73.95,0,75.809744,19.365190,99.900772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VUT,88.830000,65.38000,92.638976,92.9,59.216414,71.50,95.6,25.2,0.6125,0.523209,...,59.072419,3.786170,43.4,46.0,0.000000,59.47,0,25.719788,24.157366,96.148025
YEM,58.742400,34.19440,97.806622,65.6,35.117162,53.50,83.7,17.1,0.9953,0.562939,...,52.217281,2.807062,30.7,14.0,0.000000,37.77,0,26.718355,16.297863,87.551274
ZAF,75.470000,65.75000,94.014762,93.9,48.923554,72.60,97.5,28.3,3.8095,0.685924,...,31.476937,4.616207,85.0,43.0,0.125888,79.61,0,56.167394,30.881298,99.747741
ZMB,46.950000,28.44000,99.975388,55.5,59.216414,60.00,93.7,8.1,2.4180,0.498532,...,41.391224,4.225066,11.3,35.0,0.000000,64.64,0,27.852579,16.851228,99.599681


In [52]:
# updated SDG - COFOC expenditure mapping for all countries, manual mapping:
econ_map[2].extend(['Expenditure_on_social_protection'])
econ_map[4] = ['Expenditure_on_education']
econ_map[5].extend(['Rest_of_COFOG_not_defined'])
econ_map[9] = ['Rest_of_COFOG_not_defined']
econ_map[10].extend(['Rest_of_COFOG_not_defined', 'Expenditure_on_education'])

In [35]:
# # alternative economic indicators for countries with more complete data
# inds_econ_alt = pd.read_csv(Path('data_raw/econ_alt.csv'), index_col=[0])
# inds_econ_alt.insert(0, inds_econ.columns[0],inds_econ.iloc[:,0])
# inds_econ = inds_econ_alt

# # updated SDG - COFOG expenditure mapping for countries with more complete data
# econ_map[2].extend(['Expenditure_on_social_protection'])
# econ_map[4] = ['Expenditure_on_education']
# econ_map[5] = ['Expenditure_on_economic_affairs', 'Expenditure_on_social_protection']
# econ_map[6] = ['Expenditure_on_environment_protection', 'Expenditure_on_housing__community_amenities']
# econ_map[7] = ['Expenditure_on_economic_affairs']
# econ_map[8] = ['Expenditure_on_economic_affairs']
# econ_map[9] = ['Expenditure_on_economic_affairs']
# econ_map[10] = ['Expenditure_on_general_public_services', 'Expenditure_on_defense', 'Expenditure_on_public_order__safety', 'Expenditure_on_education', 'Expenditure_on_social_protection']
# econ_map[11] = ['Expenditure_on_public_order__safety', 'Expenditure_on_housing__community_amenities', 'Expenditure_on_recreation_culture__religion']
# econ_map[12] = ['Expenditure_on_economic_affairs', 'Expenditure_on_environment_protection']
# econ_map[13] = ['Expenditure_on_economic_affairs', 'Expenditure_on_environment_protection']
# econ_map[14] = ['Expenditure_on_economic_affairs', 'Expenditure_on_environment_protection']
# econ_map[15] = ['Expenditure_on_economic_affairs', 'Expenditure_on_environment_protection']
# econ_map[16] = ['Expenditure_on_general_public_services', 'Expenditure_on_defense', 'Expenditure_on_public_order__safety']
# econ_map[17] = ['Expenditure_on_general_public_services']

In [53]:
# shorten cofog names
# in mapping...
for key in econ_map:
    newnames = []
    for ii in econ_map[key]:    
        if ii != 'Rest_of_COFOG_not_defined':
            newnames.append(ii.split('_')[2])
        else:
            newnames.append(ii.split('_')[0].lower())
    econ_map[key] = newnames

# ... and in econ data
inds_econ.columns = [inds_econ.columns[0], 
                    *[ii.split('_')[2] for ii in inds_econ.columns[1:-1]], 
                    inds_econ.columns[-1].split('_')[0].lower()]

In [54]:
# separate indicators which will be used as undesired outputs, or not be used at all
use_inds_path = Path('data_raw/use_inds.xlsx')
use_inds = pd.read_excel(use_inds_path, usecols='A:D')
use_inds = use_inds.drop(use_inds[use_inds['ind_name'].isin(drop_inds)].index) # drop indicators we excluded above
skip_inds = use_inds['ind_name'][use_inds['use']=='no']
undes_inds = use_inds['ind_name'][use_inds['issue']=='undesired']

In [55]:
all_sdgs = range(1,18)
gms_input_folder = Path('gams_data_input/')

# define set for countries
set_c = ', '.join((inds_econ.index))

for sdg in all_sdgs:

    # define sets for inputs and outputs
    # set_i = econ_map[sdg]
    set_i = '_'.join((econ_map[sdg])) # handle sum of multiple expenses as input
    des_out = list(set(inds_map[sdg]) - set(undes_inds) - set(skip_inds)) # set operation to get desired outputs
    undes_out = list(set(inds_map[sdg]) & set(undes_inds)) #  set operation to get undesired outputs

    if len(des_out) > 0:
        set_o = ', '.join(des_out)
    else: # placeholder set if empty
        set_o = 'dm'

    if len(undes_out) > 0:
        set_o_u = ', '.join(undes_out)
    else: # placeholder set if empty
        set_o_u = 'dm'

    # write sets to .inc file
    with open(gms_input_folder / 'sets_sdg{0}.inc'.format(sdg), 'w') as text_file:
        set_string = 'set c / {0} /;\nset i / {1} /;\nset o / {2} /;\nset o_u / {3} /;'.format(set_c, set_i, set_o, set_o_u)
        text_file.write(set_string)

    # define parameters for inputs and outputs
    # placeholder column to use if no outputs or undesired outputs exist for an SDG
    dm = pd.DataFrame([1]*len(inds_econ.index), index=inds_econ.index, columns=['dm']) 

    # write inputs to .csv file
    # inds_econ.loc[:,[econ_map[sdg]]].to_csv(gms_input_folder / 'inputs_sdg{0}.csv'.format(sdg))
    inds_econ[[*econ_map[sdg]]].sum(axis=1).to_csv(gms_input_folder / 'inputs_sdg{0}.csv'.format(sdg), header=[set_i])

    # write outputs to .csv file
    if len(des_out) > 0:
        inds.loc[inds_econ.index,des_out].to_csv(gms_input_folder / 'outputs_sdg{0}.csv'.format(sdg))
    else:
        dm.to_csv(gms_input_folder / 'outputs_sdg{0}.csv'.format(sdg))

    # write undesired outputs to .csv file
    if len(undes_out) > 0:
        inds.loc[inds_econ.index,undes_out].to_csv(gms_input_folder / 'undes_outputs_sdg{0}.csv'.format(sdg))
    else:
        dm.to_csv(gms_input_folder / 'undes_outputs_sdg{0}.csv'.format(sdg))

In [ ]:
# sdg = 4
# set_i = '_and_'.join((econ_map[sdg]))
# sdg = 5
# inds_econ[[*econ_map[sdg]]].sum(axis=1)